In [1]:
import re
from typing import List

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import optuna
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

tqdm.pandas()

In [2]:
df = pd.read_csv("./data/train_folds.zip")
df_test = pd.read_csv("./data/Test.zip")

df1 = pd.read_csv("./data/train_pred_1.csv")
df1.columns = ["id", "pred_1"]
df2 = pd.read_csv("./data/train_pred_2.csv")
df2.columns = ["id", "pred_2"]
df3 = pd.read_csv("./data/train_pred_3.csv")
df3.columns = ["id", "pred_3"]
df4 = pd.read_csv("./data/train_pred_4.csv")
df4.columns = ["id", "pred_4"]
df5 = pd.read_csv("./data/train_pred_5.csv")
df5.columns = ["id", "pred_5"]

df_test1 = pd.read_csv("./data/test_pred_1.csv")
df_test1.columns = ["id", "pred_1"]
df_test2 = pd.read_csv("./data/test_pred_2.csv")
df_test2.columns = ["id", "pred_2"]
df_test3 = pd.read_csv("./data/test_pred_3.csv")
df_test3.columns = ["id", "pred_3"]
df_test4 = pd.read_csv("./data/test_pred_4.csv")
df_test4.columns = ["id", "pred_4"]
df_test5 = pd.read_csv("./data/test_pred_5.csv")
df_test5.columns = ["id", "pred_5"]

df = df.merge(df1, left_on="user_id", right_on='id', how="left")
df = df.merge(df2, left_on="user_id", right_on='id', how="left")
df = df.merge(df3, left_on="user_id", right_on='id', how="left")
df = df.merge(df4, left_on="user_id", right_on='id', how="left")
df = df.merge(df5, left_on="user_id", right_on='id', how="left")

df_test = df_test.merge(df_test1, left_on="user_id", right_on='id', how="left")
df_test = df_test.merge(df_test2, left_on="user_id", right_on='id', how="left")
df_test = df_test.merge(df_test3, left_on="user_id", right_on='id', how="left")
df_test = df_test.merge(df_test4, left_on="user_id", right_on='id', how="left")
df_test = df_test.merge(df_test5, left_on="user_id", right_on='id', how="left")

df.head()

h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\pandas\core\frame.py:9186: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,...,id_x,pred_1,id_y,pred_2,id_x,pred_3,id_y,pred_4,id,pred_5
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,...,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000192,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000734,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.000458,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.004463,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,0.008806
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.869597,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.620179,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.627454,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.607547,00000cb4a5d760de88fecb38e2f71b7bec52e834,0.879773
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,...,00001654a9d9f96303d9969d0a4a851714a4bb57,0.282249,00001654a9d9f96303d9969d0a4a851714a4bb57,0.148125,00001654a9d9f96303d9969d0a4a851714a4bb57,0.136944,00001654a9d9f96303d9969d0a4a851714a4bb57,0.132337,00001654a9d9f96303d9969d0a4a851714a4bb57,0.305990
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,...,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000246,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000508,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.000498,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.004372,00001dd6fa45f7ba044bd5d84937be464ce78ac2,0.005401
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,...,000028d9e13a595abe061f9b58f3d76ab907850f,0.104483,000028d9e13a595abe061f9b58f3d76ab907850f,0.025337,000028d9e13a595abe061f9b58f3d76ab907850f,0.021932,000028d9e13a595abe061f9b58f3d76ab907850f,0.021137,000028d9e13a595abe061f9b58f3d76ab907850f,0.116135


In [9]:
class Run:
    def __init__(self, model_class, params_builder, fit_params_builder=None):
        self._params_builder = params_builder
        self._model_class = model_class
        self._fit_params_builder = fit_params_builder
        
    def __call__(self, trial):
        fold = 0
        useful_cols = ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']

        xtrain = df[df['kfold'] != fold].reset_index(drop=True)
        xvalid = df[df['kfold'] == fold].reset_index(drop=True)

        ytrain = xtrain['CHURN']
        yvalid = xvalid['CHURN']

        xtrain = xtrain[useful_cols]
        xvalid = xvalid[useful_cols]

        params = self._params_builder(trial)

        model = self._model_class(**params)

        fit_params = self._fit_params_builder(xvalid, yvalid) if self._fit_params_builder is not None else {}
        model.fit(xtrain, ytrain, **fit_params)
        
        preds_valid = model.predict_proba(xvalid)[:, 1]
        score = roc_auc_score(yvalid, preds_valid)
        return score

In [10]:
def xgb_fit_params_builder(xvalid, yvalid):
    return {
        'early_stopping_rounds': 300, 
        'eval_set': [(xvalid, yvalid)], 
        'verbose': 1000
    }

def xgb_params_buidler(trial):
    return {
        'learning_rate': trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
        'reg_lambda': trial.suggest_loguniform("reg_lambda", 1e-8, 100.0),
        'reg_alpha': trial.suggest_loguniform("reg_alpha", 1e-8, 100.0),
        'subsample': trial.suggest_float("subsample", 0.1, 1.0),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1.0),
        'max_depth': trial.suggest_int("max_depth", 1, 7),
        'random_state': 1,
        'n_estimators': 7000,
        'tree_method': 'gpu_hist',
        'gpu_id': 0,
        'predictor': "gpu_predictor",
    }

xgb_study = optuna.create_study(direction="maximize")
xgb_study.optimize(Run(XGBClassifier, xgb_params_buidler, xgb_fit_params_builder))
xgb_study.best_params

[I 2021-09-21 21:35:07,191] A new study created in memory with name: no-name-1f69db01-cf10-42ae-9614-aad5318c7dfb
h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:35:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.60521
[1000]	validation_0-logloss:0.25193
[1339]	validation_0-logloss:0.25196


[I 2021-09-21 21:35:24,674] Trial 0 finished with value: 0.9313452479711448 and parameters: {'learning_rate': 0.14186620982968798, 'reg_lambda': 3.55898370381895e-08, 'reg_alpha': 3.0013165259137945e-06, 'subsample': 0.8280581713496301, 'colsample_bytree': 0.1718069127762427, 'max_depth': 3}. Best is trial 0 with value: 0.9313452479711448.
h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:35:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.59123
[339]	validation_0-logloss:0.25371


[I 2021-09-21 21:35:33,615] Trial 1 finished with value: 0.9314233350334096 and parameters: {'learning_rate': 0.16485306189596738, 'reg_lambda': 1.758278917718893e-08, 'reg_alpha': 1.6053958496850559e-06, 'subsample': 0.5107660544364498, 'colsample_bytree': 0.6650741481584964, 'max_depth': 7}. Best is trial 1 with value: 0.9314233350334096.
h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[I 2021-09-21 21:48:55,871] Trial 44 finished with value: 0.9315272373011102 and parameters: {'learning_rate': 0.012279294564643916, 'reg_lambda': 3.5341083287733105e-06, 'reg_alpha': 2.3190508055586554e-05, 'subsample': 0.9271952162018878, 'colsample_bytree': 0.9955235948592415, 'max_depth': 4}. Best is trial 44 with value: 0.9315272373011102.
h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:48:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.68479
[955]	validation_0-logloss:0.25154


[I 2021-09-21 21:49:12,371] Trial 45 finished with value: 0.9315219389465543 and parameters: {'learning_rate': 0.012480171691547024, 'reg_lambda': 1.9255881280874044e-07, 'reg_alpha': 4.676084599935961e-05, 'subsample': 0.9323656164321777, 'colsample_bytree': 0.974954876981782, 'max_depth': 5}. Best is trial 44 with value: 0.9315272373011102.
h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:49:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.68225


KeyboardInterrupt: 

In [11]:
xgb_study.best_params

{'learning_rate': 0.012279294564643916,
 'reg_lambda': 3.5341083287733105e-06,
 'reg_alpha': 2.3190508055586554e-05,
 'subsample': 0.9271952162018878,
 'colsample_bytree': 0.9955235948592415,
 'max_depth': 4}

In [13]:
def lgb_fit_params_builder(xvalid, yvalid):
    return {
        'early_stopping_rounds': 300, 
        'eval_set': [(xvalid, yvalid)], 
        'verbose': 1000
    }

def lgb_params_buidler(trial):
    return {
        'learning_rate': trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
        'reg_lambda': trial.suggest_loguniform("reg_lambda", 1e-8, 100.0),
        'reg_alpha': trial.suggest_loguniform("reg_alpha", 1e-8, 100.0),
        'subsample': trial.suggest_float("subsample", 0.1, 1.0),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1.0),
        'max_depth': trial.suggest_int("max_depth", 1, 7),       
        'random_state': 42,
        'n_estimators': 7000,
    }

lgb_study = optuna.create_study(direction="maximize")
lgb_study.optimize(Run(LGBMClassifier, lgb_params_buidler, lgb_fit_params_builder), n_trials=50, timeout=60*60*2)
lgb_study.best_params

[I 2021-09-21 21:51:16,256] A new study created in memory with name: no-name-8c698ff2-9878-49b2-aba6-445619a23024


Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[270]	valid_0's binary_logloss: 0.251629


[I 2021-09-21 21:51:35,798] Trial 0 finished with value: 0.9314616765954945 and parameters: {'learning_rate': 0.13921524861958795, 'reg_lambda': 2.0757373247040976e-06, 'reg_alpha': 1.526233335372189e-06, 'subsample': 0.6567876461306602, 'colsample_bytree': 0.4673330568614352, 'max_depth': 3}. Best is trial 0 with value: 0.9314616765954945.


Training until validation scores don't improve for 300 rounds
[1000]	valid_0's binary_logloss: 0.251674
[2000]	valid_0's binary_logloss: 0.251658
[3000]	valid_0's binary_logloss: 0.251647
[4000]	valid_0's binary_logloss: 0.251639
[5000]	valid_0's binary_logloss: 0.251633
[6000]	valid_0's binary_logloss: 0.25163
[7000]	valid_0's binary_logloss: 0.251627
Did not meet early stopping. Best iteration is:
[7000]	valid_0's binary_logloss: 0.251627


[I 2021-09-21 21:54:42,111] Trial 1 finished with value: 0.9314595016803876 and parameters: {'learning_rate': 0.10856708304499599, 'reg_lambda': 1.626204828063972e-05, 'reg_alpha': 0.004259197252294888, 'subsample': 0.29050794139033503, 'colsample_bytree': 0.7100857431202984, 'max_depth': 1}. Best is trial 0 with value: 0.9314616765954945.


...

Training until validation scores don't improve for 300 rounds
[1000]	valid_0's binary_logloss: 0.251596
Early stopping, best iteration is:
[1194]	valid_0's binary_logloss: 0.251595


[I 2021-09-21 22:32:04,357] Trial 48 finished with value: 0.9314808812366564 and parameters: {'learning_rate': 0.016236669719302658, 'reg_lambda': 0.058055036190272205, 'reg_alpha': 7.174381810144883, 'subsample': 0.9598544109190894, 'colsample_bytree': 0.4718766568406161, 'max_depth': 5}. Best is trial 23 with value: 0.9315240997672434.


Training until validation scores don't improve for 300 rounds
[1000]	valid_0's binary_logloss: 0.251579
Early stopping, best iteration is:
[760]	valid_0's binary_logloss: 0.251564


[I 2021-09-21 22:32:47,060] Trial 49 finished with value: 0.9314952251381826 and parameters: {'learning_rate': 0.02741185721319138, 'reg_lambda': 0.4090340272588048, 'reg_alpha': 0.24146580217433208, 'subsample': 0.5636273347558117, 'colsample_bytree': 0.7939459481200335, 'max_depth': 4}. Best is trial 23 with value: 0.9315240997672434.


{'learning_rate': 0.03924735805719372,
 'reg_lambda': 2.0139848865528562,
 'reg_alpha': 2.380837889033024,
 'subsample': 0.6977006896659559,
 'colsample_bytree': 0.9408555739613342,
 'max_depth': 7}

In [14]:
def rf_params_buidler(trial):
    return {
        'max_depth': trial.suggest_int('rf_max_depth', 2, 32, log=True),
        'max_features': trial.suggest_categorical('rf_max_features', ["auto", "sqrt", "log2"]),
        'class_weight': trial.suggest_categorical('rf_class_weight', ['balanced', 'balanced_subsample', None]),
        'n_estimators': 100,
        'n_jobs': -1,
        'random_state': 42,
        'verbose': 1,
    }

rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(Run(RandomForestClassifier, rf_params_buidler), n_trials=50, timeout=60*60*2)
rf_study.best_params

[I 2021-09-21 22:32:47,086] A new study created in memory with name: no-name-885c2d83-2f91-4559-baea-7401713eaca7
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   52.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.3min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    1.5s finished
[I 2021-09-21 22:35:09,335] Trial 0 finished with value: 0.9310460195260559 and parameters: {'rf_max_depth': 15, 'rf_max_features': 'auto', 'rf_class_weight': 'balanced'}. Best is trial 0 with value: 0.9310460195260559.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   48.5s finished
[Parallel(n_jobs=8)]: Using

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.8s finished
[I 2021-09-21 22:50:30,990] Trial 12 finished with value: 0.9314952617589731 and parameters: {'rf_max_depth': 9, 'rf_max_features': 'auto', 'rf_class_weight': None}. Best is trial 10 with value: 0.9315042639819253.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.7s finished
[I 2021-09-21 22:51:49,571] Trial 13 finished with value: 0.9314914418365392 and parameters: {'rf_max_depth': 7, 'rf_max_features': 'auto', 'rf_class_weight': None}. Best is trial 10 with value: 0.9315042639819253.
[Parallel(n_jo

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.5min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    3.5s finished
[I 2021-09-21 23:15:31,893] Trial 25 finished with value: 0.9292255597507528 and parameters: {'rf_max_depth': 31, 'rf_max_features': 'log2', 'rf_class_weight': None}. Best is trial 10 with value: 0.9315042639819253.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.7min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.6s
[Par

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.6s finished
[I 2021-09-21 23:38:31,054] Trial 37 finished with value: 0.9313636209358054 and parameters: {'rf_max_depth': 5, 'rf_max_features': 'auto', 'rf_class_weight': None}. Best is trial 10 with value: 0.9315042639819253.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.4min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.7s finished
[I 2021-09-21 23:39:57,945] Trial 38 finished with value: 0.9315039455962556 and parameters: {'rf_max_depth': 8, 'rf_max_features': 'auto', 'rf_class_weight': None}. Best is trial 10 with value: 0.9315042639819253.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent wor

{'rf_max_depth': 10, 'rf_max_features': 'auto', 'rf_class_weight': None}

In [16]:
def cb_fit_params_builder(xvalid, yvalid):
    return {
        'early_stopping_rounds': 300, 
        'eval_set': [(xvalid, yvalid)], 
        'verbose': 1000
    }

def cb_params_buidler(trial):    
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        'random_state': 1
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    return param

cb_study = optuna.create_study(direction="maximize")
cb_study.optimize(Run(CatBoostClassifier, cb_params_buidler, cb_fit_params_builder), 
                  n_trials=50,
                  timeout=60*60*2,
                  gc_after_trial=True,
                 show_progress_bar=True)

cb_study.best_params

[I 2021-09-22 09:01:48,660] A new study created in memory with name: no-name-0b3a7837-eeb8-40b2-8f7a-de634477dba0
h:\projects\zindi-expresso-churn-prediction-challenge\venv\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

0:	learn: 0.6796327	test: 0.6796633	best: 0.6796633 (0)	total: 330ms	remaining: 5m 29s
999:	learn: 0.2620423	test: 0.2623776	best: 0.2623776 (999)	total: 1m 56s	remaining: 0us

bestTest = 0.2623776244
bestIteration = 999

[I 2021-09-22 09:03:47,432] Trial 0 finished with value: 0.9290530728103573 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.016969110504932546, 'depth': 8, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.1427344213620881}. Best is trial 0 with value: 0.9290530728103573.
0:	learn: 0.6796327	test: 0.6796633	best: 0.6796633 (0)	total: 352ms	remaining: 5m 51s
999:	learn: 0.2532269	test: 0.2535496	best: 0.2535496 (999)	total: 5m 17s	remaining: 0us

bestTest = 0.2535496199
bestIteration = 999

[I 2021-09-22 09:09:08,798] Trial 1 finished with value: 0.9309186157601178 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.03196534993902082, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subs

Learning rate set to 0.199658
0:	learn: 0.5219112	test: 0.5215581	best: 0.5215581 (0)	total: 476ms	remaining: 7m 55s
999:	learn: 0.2512459	test: 0.2518865	best: 0.2518863 (990)	total: 6m 39s	remaining: 0us

bestTest = 0.2518862935
bestIteration = 990

Shrink model to first 991 iterations.
[I 2021-09-22 10:16:33,755] Trial 15 finished with value: 0.9313792672744493 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.0805123693592737, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 15 with value: 0.9313792672744493.
Learning rate set to 0.199658
0:	learn: 0.5219112	test: 0.5215581	best: 0.5215581 (0)	total: 436ms	remaining: 7m 15s
999:	learn: 0.2514830	test: 0.2521242	best: 0.2521242 (999)	total: 7m 46s	remaining: 0us

bestTest = 0.2521241525
bestIteration = 999

[I 2021-09-22 10:24:24,284] Trial 16 finished with value: 0.9312289533584264 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.07552767562425587, 'depth': 4, 'boosting_t

{'objective': 'Logloss',
 'colsample_bylevel': 0.0805123693592737,
 'depth': 10,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'MVS'}

In [ ]:
def cb_fit_params_builder(xvalid, yvalid):
    return {
        'early_stopping_rounds': 300, 
        'eval_set': [(xvalid, yvalid)], 
        'verbose': 1000
    }

def cb_params_buidler(trial):    
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        'random_state': 1
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    return param

cb_study = optuna.create_study(direction="maximize")
cb_study.optimize(Run(CatBoostClassifier, cb_params_buidler, cb_fit_params_builder), 
                  n_trials=50,
                  timeout=60*60*2,
                  gc_after_trial=True,
                 show_progress_bar=True)

cb_study.best_params